In [1]:
import requests
import pandas as pd
import time
import os

bearer_token = os.environ.get("TWITTER_BEARER_TOKEN")
user_id = "1498510162004361220"  # Replace with the actual user ID

url = f"https://api.twitter.com/2/users/{user_id}/tweets"

headers = {
    "Authorization": f"Bearer {bearer_token}"
}

start_time = "2022-03-22T00:00:00Z"  # Start of the day in UTC
end_time = "2022-03-26T23:59:59Z"    # End of the day in UTC

params = {
    "max_results": 100,  # Maximum number of tweets per request
    "tweet.fields": "created_at",  # Request the creation date of the tweet
    #"start_time": start_time,
    #"end_time": end_time
}

csv_file = "tweets.csv"  # Path to the CSV file
file_exists = os.path.isfile(csv_file)  # Check if the CSV file already exists

next_token = None
delay = 360  # Initial delay in seconds to stay well within the rate limit

dfs = []

while True:
    if next_token:
        params['pagination_token'] = next_token  # Set the pagination token

    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        tweets_data = response.json()
        tweets = tweets_data.get('data', [])
        print(tweets_data)

        # Convert the current batch of tweets to a DataFrame
        df = pd.DataFrame(tweets)
        
        # Append the DataFrame to the CSV file
        df.to_csv(csv_file, mode='a', index=False, header=not file_exists)
        dfs.append(df)
        file_exists = True  # Ensure header is not written again
        
        print(f"Appended {len(tweets)} tweets to list")

        # Check if there's another page of results
        next_token = tweets_data.get('meta', {}).get('next_token', None)
        if not next_token:
            break  # Exit the loop if there are no more tweets to fetch

        # Delay before the next request
        time.sleep(delay)
        
    elif response.status_code == 429:
        print("Rate limit exceeded. Waiting before retrying...")
        # Extract the time to wait from the headers if available
        reset_time = int(response.headers.get('x-rate-limit-reset', time.time() + 900)) - int(time.time())
        delay = max(reset_time, 2)  # Ensure delay is at least 2 seconds
        time.sleep(delay)
    else:
        print(f"Failed to fetch tweets: {response.status_code} - {response.text}")
        break

print("Data collection completed.")


Failed to fetch tweets: 401 - {
  "title": "Unauthorized",
  "type": "about:blank",
  "status": 401,
  "detail": "Unauthorized"
}
Data collection completed.
